### Notebook for concept detection in neural network

In [1]:
%load_ext autoreload
%autoreload 2

In [10]:
import numpy as np
import os
import sys
from tqdm import tqdm
import json
import random
import string

sys.path.append(os.path.abspath(os.path.join(os.path.pardir, 'src')))

import concepts
import env
from policy import ActorCriticNet

model_name = "net"
session_name = "starship"
board_size = 5
board_name = f'{board_size}x{board_size}'

agents_to_sample = [0, 150, 300, 450, 600, 750, 900, 1050, 1200, 1500, 1800]

full_model_path = f"../models/saved_sessions/board_size_{board_size}/{session_name}/"

CONCEPT_FUNC = concepts.capture_stones

CONCEPT_NAME = concepts.capture_stones.__name__

CASES_TO_SAMPLE = 1000 # 25000


In [11]:
def load_model(full_name, model_name, epoch):
    model_path = full_name + model_name + "_" + str(epoch) + ".keras"
    model = ActorCriticNet(board_size, model_path)
    return model

agents = [load_model(full_model_path, model_name, epoch) for epoch in agents_to_sample]

In [13]:
def play_match(agents: list[ActorCriticNet], board_size, concept_function):
    go_env = env.GoEnv(board_size)
    state = go_env.reset()

    player_to_start = 1 if np.random.random() > 0.5 else 0
    current_player = player_to_start

    total_moves = board_size * board_size * 4
    moves = 0
    random_moves = False

    positive_cases = []
    negative_cases = []

    sample_ratio = 0.2
    alpha = 0.5 # The higher, the more unique games. 0 = argmax, 1 = distribution 

    game_over = False

    while not game_over:
        if moves == total_moves:
            break
        # Remove array index 3 and 5 from the current state making it an shape of (4, 5, 5)
        state_copy = np.delete(state, [3, 5], axis=0)
        
        if np.random.random() < sample_ratio:
            if concept_function(state):
                positive_cases.append(state_copy)
            else:
                negative_cases.append(state_copy)

        if random_moves:
            action = go_env.uniform_random_action()
        else:
            action = agents[current_player].best_action(state, alpha=alpha)

        state, _, game_over, _ = go_env.step(action)
        moves += 1

        current_player = (current_player + 1) % 2
    
    return positive_cases, negative_cases

positive_cases = []
negative_cases = []

positive_bar = tqdm(total=CASES_TO_SAMPLE, desc="Positive cases")

while len(positive_cases) < CASES_TO_SAMPLE or len(negative_cases) < CASES_TO_SAMPLE:
    for i in range(len(agents)):
        for j in range(i + 1, len(agents)):
            pos, neg = play_match([agents[i], agents[j]], board_size, CONCEPT_FUNC)
            positive_cases.extend(pos)
            negative_cases.extend(neg)
            positive_bar.update(len(pos))

positive_cases = positive_cases[:CASES_TO_SAMPLE]
negative_cases = negative_cases[:CASES_TO_SAMPLE]

# Create numpy arrays
positive_cases = np.array(positive_cases)
negative_cases = np.array(negative_cases)

Positive cases:  11%|█▏        | 113/1000 [00:13<01:47,  8.22it/s]
Positive cases: 1078it [01:37, 10.18it/s]                         

In [14]:
# Print shapes
print("Positive cases: ", positive_cases.shape)
print("Negative cases: ", negative_cases.shape)

Positive cases:  (1000, 4, 5, 5)
Negative cases:  (1000, 4, 5, 5)


In [15]:
# Test if there are any duplicates accross the two sets
for i in range(len(positive_cases)):
    for j in range(len(negative_cases)):
        if np.array_equal(positive_cases[i], negative_cases[j]):
            print("Duplicate found!")
            print(positive_cases[i])
            print(negative_cases[j])
            break

In [16]:
# Find the number of any duplicates within the positive cases
duplicate_count = 0
for i in range(len(positive_cases)):
    for j in range(i + 1, len(positive_cases)):
        if np.array_equal(positive_cases[i], positive_cases[j]):
            duplicate_count += 1
            break

print("Duplicate count in positive cases: ", duplicate_count)

# Find the number of any duplicates within the negative cases
duplicate_count = 0
for i in range(len(negative_cases)):
    for j in range(i + 1, len(negative_cases)):
        if np.array_equal(negative_cases[i], negative_cases[j]):
            duplicate_count += 1
            break

print("Duplicate count in negative: ", duplicate_count)

Duplicate count in positive cases:  0
Duplicate count in negative:  101


Positive cases: 1078it [01:53, 10.18it/s]

In [ ]:
# Print all the positive cases
for i in range(len(positive_cases)):
    print(f"Positive case {i}:")
    print(positive_cases[i])
    print()

In [20]:
# Positions to consider are 80% of the total positions
POSITIONS_TO_CONSIDER = int(0.8 * positive_cases.shape[0] * 2)
print(f"Positions to consider: {POSITIONS_TO_CONSIDER}")
#POSITIONS_TO_CONSIDER = 4000 #40000
VALIDATION_POSITIONS = 10000 #10000

BINARY = True

Positions to consider: 1600


In [21]:
# First test if the concept can be regressed form the inputs
name = "input"
all_cases = np.concatenate([positive_cases, negative_cases])
all_labels = [1] * positive_cases.shape[0] + [0] * negative_cases.shape[0]
all_labels = np.array(all_labels)
shuffled_indices = np.arange(all_labels.shape[0])

np.random.shuffle(shuffled_indices)

all_cases = all_cases[shuffled_indices]
all_labels = all_labels[shuffled_indices]

points = all_cases.reshape(all_cases.shape[0], -1)

# Use the regression
score = concepts.perform_regression(
    points=points[:POSITIONS_TO_CONSIDER], 
    targets=all_labels[:POSITIONS_TO_CONSIDER], 
    validation_points=points[POSITIONS_TO_CONSIDER:], 
    validation_targets=all_labels[POSITIONS_TO_CONSIDER:], 
    is_binary=BINARY
)

print("Regression score: ", score)

# Remove the files if they exist
if os.path.exists("../concept_presences/{}/{}/{}/".format(board_name, session_name, CONCEPT_NAME)):
    # Test if epoch folder exists
    if os.path.exists("../concept_presences/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, name)):
        # Remove all files in the epoch folder
        # Fist save the 
        for file in os.listdir("../concept_presences/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, name)):
            os.remove("../concept_presences/{}/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, name, file))

os.makedirs("../concept_presences", exist_ok=True)
os.makedirs("../concept_presences/{}".format(board_name), exist_ok=True)
os.makedirs("../concept_presences/{}/{}".format(board_name, session_name), exist_ok=True)
os.makedirs("../concept_presences/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME), exist_ok=True)
os.makedirs("../concept_presences/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, name), exist_ok=True)

random_suffix = ''.join(random.choices(string.ascii_uppercase + string.digits, k=10))

# Save concept presences in json file
with open("../concept_presences/{}/{}/{}/{}/{}.json".format(board_name, session_name, CONCEPT_NAME, name, random_suffix), "w") as f:
    json.dump(score, f)

Epoch 1/50
50/50 [==============================] - 0s 3ms/step - loss: 0.7942 - val_loss: 0.7929
Epoch 2/50
50/50 [==============================] - 0s 2ms/step - loss: 0.7901 - val_loss: 0.7884
Epoch 3/50
50/50 [==============================] - 0s 2ms/step - loss: 0.7867 - val_loss: 0.7846
Epoch 4/50
50/50 [==============================] - 0s 2ms/step - loss: 0.7835 - val_loss: 0.7811
Epoch 5/50
50/50 [==============================] - 0s 2ms/step - loss: 0.7806 - val_loss: 0.7780
Epoch 6/50
50/50 [==============================] - 0s 2ms/step - loss: 0.7780 - val_loss: 0.7750
Epoch 7/50
50/50 [==============================] - 0s 1ms/step - loss: 0.7752 - val_loss: 0.7721
Epoch 8/50
50/50 [==============================] - 0s 1ms/step - loss: 0.7725 - val_loss: 0.7693
Epoch 9/50
50/50 [==============================] - 0s 2ms/step - loss: 0.7700 - val_loss: 0.7666
Epoch 10/50
50/50 [==============================] - 0s 1ms/step - loss: 0.7676 - val_loss: 0.7640
Epoch 11/50
50/50 [

In [22]:
epochs_to_look_at = [0, 150, 300, 450, 600, 750, 900, 1050, 1200, 1500, 1800]

for epoch in epochs_to_look_at:
    path = full_model_path + model_name + "_" + str(epoch) + ".keras"
    model = ActorCriticNet(board_size, path)

    # Will have a length equal to the sum of the numer of rows in the positive and negative cases arrays
    # And will contain 1s for positive cases and 0s for negative cases
    # Is used as labels/targets for the regression
    all_cases = np.concatenate([positive_cases, negative_cases])
    all_labels = [1] * positive_cases.shape[0] + [0] * negative_cases.shape[0]
    all_labels = np.array(all_labels)
    shuffled_indices = np.arange(all_labels.shape[0])

    np.random.shuffle(shuffled_indices)

    all_cases = all_cases[shuffled_indices]
    all_labels = all_labels[shuffled_indices]

    concept_presences = {}
    
    outputs = model.get_all_activation_values(all_cases)

    # Merge outputs
    merged_outputs = []
    for output_batch in outputs:
        for i, output_layer in enumerate(output_batch):
            if len(merged_outputs) <= i:
                merged_outputs.append([])
            merged_outputs[i].extend(output_layer)

    for i, layer_output in enumerate(merged_outputs):
        merged_outputs[i] = np.array(merged_outputs[i])
    
    outputs = merged_outputs

    # Perform regression
    concept_presence_per_layer = []
    for (i, output) in enumerate(outputs):
        points = output.reshape((output.shape[0], np.prod(output.shape[1:])))
        # So one has (n, k) samples where n is the number of positions, and k is the total number of activation values in layer i.
        print(f"Performing regression for layer {i}")
        score = concepts.perform_regression(
            points=points[:POSITIONS_TO_CONSIDER], 
            targets=all_labels[:POSITIONS_TO_CONSIDER], 
            validation_points=points[POSITIONS_TO_CONSIDER:], 
            validation_targets=all_labels[POSITIONS_TO_CONSIDER:], 
            is_binary=BINARY
        )
        concept_presence_per_layer.append(score)

        print(f"The presence of {CONCEPT_NAME} in resblock {i} is {score}")
    concept_presences[CONCEPT_NAME] = concept_presence_per_layer

    # Remove the files if they exist
    if os.path.exists("../concept_presences/{}/{}/{}/".format(board_name, session_name, CONCEPT_NAME)):
        # Test if epoch folder exists
        if os.path.exists("../concept_presences/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, epoch)):
            # Remove all files in the epoch folder
            # Fist save the 
            for file in os.listdir("../concept_presences/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, epoch)):
                os.remove("../concept_presences/{}/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, epoch, file))

    os.makedirs("../concept_presences", exist_ok=True)
    os.makedirs("../concept_presences/{}".format(board_name), exist_ok=True)
    os.makedirs("../concept_presences/{}/{}".format(board_name, session_name), exist_ok=True)
    os.makedirs("../concept_presences/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME), exist_ok=True)
    os.makedirs("../concept_presences/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, epoch), exist_ok=True)

    random_suffix = ''.join(random.choices(string.ascii_uppercase + string.digits, k=10))

    # Save concept presences in json file
    with open("../concept_presences/{}/{}/{}/{}/{}.json".format(board_name, session_name, CONCEPT_NAME, epoch, random_suffix), "w") as f:
        json.dump(concept_presences[CONCEPT_NAME], f)




Getting activation outputs: 100%|██████████| 63/63 [00:00<00:00, 234.66it/s]


Performing regression for layer 0
Epoch 1/50
50/50 [==============================] - 1s 7ms/step - loss: 0.9948 - val_loss: 0.9807
Epoch 2/50
50/50 [==============================] - 0s 1ms/step - loss: 0.9707 - val_loss: 0.9571
Epoch 3/50
50/50 [==============================] - 0s 1ms/step - loss: 0.9480 - val_loss: 0.9349
Epoch 4/50
50/50 [==============================] - 0s 1ms/step - loss: 0.9267 - val_loss: 0.9143
Epoch 5/50
50/50 [==============================] - 0s 2ms/step - loss: 0.9071 - val_loss: 0.8950
Epoch 6/50
50/50 [==============================] - 0s 2ms/step - loss: 0.8884 - val_loss: 0.8768
Epoch 7/50
50/50 [==============================] - 0s 1ms/step - loss: 0.8711 - val_loss: 0.8599
Epoch 8/50
50/50 [==============================] - 0s 1ms/step - loss: 0.8546 - val_loss: 0.8440
Epoch 9/50
50/50 [==============================] - 0s 1ms/step - loss: 0.8390 - val_loss: 0.8286
Epoch 10/50
50/50 [==============================] - 0s 1ms/step - loss: 0.8243 - va

Getting activation outputs: 100%|██████████| 63/63 [00:00<00:00, 284.47it/s]

Performing regression for layer 0
Epoch 1/50


50/50 [==============================] - 0s 3ms/step - loss: 0.9827 - val_loss: 0.9576
Epoch 2/50
50/50 [==============================] - 0s 1ms/step - loss: 0.9581 - val_loss: 0.9367
Epoch 3/50
50/50 [==============================] - 0s 1ms/step - loss: 0.9392 - val_loss: 0.9207
Epoch 4/50
50/50 [==============================] - 0s 1ms/step - loss: 0.9237 - val_loss: 0.9063
Epoch 5/50
50/50 [==============================] - 0s 1ms/step - loss: 0.9098 - val_loss: 0.8936
Epoch 6/50
50/50 [==============================] - 0s 1ms/step - loss: 0.8969 - val_loss: 0.8815
Epoch 7/50
50/50 [==============================] - 0s 1ms/step - loss: 0.8849 - val_loss: 0.8699
Epoch 8/50
50/50 [==============================] - 0s 1ms/step - loss: 0.8731 - val_loss: 0.8585
Epoch 9/50
50/50 [==============================] - 0s 1ms/step - loss: 0.8617 - val_loss: 0.8476
Epoch 10/50
50/50 [==============================] - 0s 1ms/step - loss: 0.8510 - val_loss: 0.8377
Epoch 11/50
50/50 [===========

...

Getting activation outputs: 100%|██████████| 63/63 [00:00<00:00, 278.16it/s]


Performing regression for layer 0
Epoch 1/50
50/50 [==============================] - 0s 3ms/step - loss: 1.0206 - val_loss: 1.0031
Epoch 2/50
50/50 [==============================] - 0s 1ms/step - loss: 0.9662 - val_loss: 0.9688
Epoch 3/50
50/50 [==============================] - 0s 1ms/step - loss: 0.9286 - val_loss: 0.9430
Epoch 4/50
50/50 [==============================] - 0s 1ms/step - loss: 0.9007 - val_loss: 0.9235
Epoch 5/50
50/50 [==============================] - 0s 1ms/step - loss: 0.8786 - val_loss: 0.9072
Epoch 6/50
50/50 [==============================] - 0s 1ms/step - loss: 0.8607 - val_loss: 0.8915
Epoch 7/50
50/50 [==============================] - 0s 1ms/step - loss: 0.8451 - val_loss: 0.8785
Epoch 8/50
50/50 [==============================] - 0s 1ms/step - loss: 0.8317 - val_loss: 0.8663
Epoch 9/50
50/50 [==============================] - 0s 1ms/step - loss: 0.8194 - val_loss: 0.8544
Epoch 10/50
50/50 [==============================] - 0s 1ms/step - loss: 0.8081 - va

Getting activation outputs: 100%|██████████| 63/63 [00:00<00:00, 282.40it/s]

Performing regression for layer 0
Epoch 1/50


50/50 [==============================] - 0s 3ms/step - loss: 1.0906 - val_loss: 1.0816
Epoch 2/50
50/50 [==============================] - 0s 1ms/step - loss: 1.0231 - val_loss: 1.0222
Epoch 3/50
50/50 [==============================] - 0s 1ms/step - loss: 0.9751 - val_loss: 0.9779
Epoch 4/50
50/50 [==============================] - 0s 1ms/step - loss: 0.9389 - val_loss: 0.9440
Epoch 5/50
50/50 [==============================] - 0s 1ms/step - loss: 0.9103 - val_loss: 0.9163
Epoch 6/50
50/50 [==============================] - 0s 1ms/step - loss: 0.8872 - val_loss: 0.8935
Epoch 7/50
50/50 [==============================] - 0s 1ms/step - loss: 0.8682 - val_loss: 0.8750
Epoch 8/50
50/50 [==============================] - 0s 1ms/step - loss: 0.8515 - val_loss: 0.8593
Epoch 9/50
50/50 [==============================] - 0s 1ms/step - loss: 0.8374 - val_loss: 0.8447
Epoch 10/50
50/50 [==============================] - 0s 1ms/step - loss: 0.8249 - val_loss: 0.8328
Epoch 11/50
50/50 [===========